[구글 코랩(Colab)에서 실행하기](https://colab.research.google.com/github/lovedlim/bigdata_analyst_cert_v2/blob/main/part4/ch9/p9_type1.ipynb)

## 작업형1

### 문제1

### 문제1

In [ ]:
# 방법1
import pandas as pd
df = pd.read_csv("loan.csv")
# df = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert_v2/main/part4/ch9/loan.csv")

# 1) 총대출액 컬럼 생성
df['총대출액'] = df['신용대출'] + df['담보대출']

# 2) 지역코드와 성별별로 총대출액 합계 계산
grouped = df.groupby(['지역코드', '성별'])['총대출액'].sum().unstack()

# 3) 성별 간 총대출액 차이가 가장 큰 지역코드 찾기 (절대값 사용)
grouped['차이'] = abs(grouped[1] - grouped[2])
result = grouped['차이'].idxmax()
result

4100000278

In [ ]:
# 방법2
import pandas as pd
df = pd.read_csv("loan.csv")
# df = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert_v2/main/part4/ch9/loan.csv")

# 1) 총대출액 컬럼 생성
df['총대출액'] = df['신용대출'] + df['담보대출']

# 2) 지역코드와 성별별로 총대출액 합계 계산
pivot_result = df.pivot_table(index='지역코드', columns='성별', values='총대출액', aggfunc='sum')

# 3) 성별 간 총대출액 차이가 가장 큰 지역코드 찾기 (절대값 사용)
pivot_result['차이'] = abs(pivot_result[1] - pivot_result[2])
result = pivot_result['차이'].idxmax()
result

4100000278

### 문제2

In [ ]:
# 방법1
import pandas as pd
df = pd.read_csv("crime.csv")
# df = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert_v2/main/part4/ch9/crime.csv")

# 1) "발생건수"와 "검거건수"를 따로 분리
cond1 = df["구분"] == "발생건수"
cond2 = df["구분"] == "검거건수"
df1 = df[cond1].iloc[:, 2:] # 발생건수만 가져오기
df2 = df[cond2].iloc[:, 2:] # 검거건수만 가져오기

# 2) 검거율 계산 (검거건수 / 발생건수)
df1 = df1.reset_index(drop=True)
df2 = df2.reset_index(drop=True)
df3 = df2 / df1

# 3) 각 연도에서 검거율이 가장 높은 범죄유형 찾기
listbox = df3.idxmax(axis=1)

# 4) 가장 높은 검거율을 기록한 범죄유형의 검거건수 가져오기
result = 0
for index, item in enumerate(listbox):
    result = result + df2.loc[index, item]

result

7799

In [ ]:
# 방법2
import pandas as pd
df = pd.read_csv("crime.csv")
# df = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert_v2/main/part4/ch9/crime.csv")

# 1. 데이터를 길게 변환
df_long = pd.melt(df, id_vars=["연도", "구분"], var_name="범죄유형", value_name="건수")

# 2. 피벗(pivot)하여 발생건수와 검거건수를 한 행에 모으기:
df_pivot = df_long.pivot_table(index=["연도", "범죄유형"], columns="구분", values="건수").reset_index()

# 3. 검거율 계산
df_pivot["검거율"] = df_pivot["검거건수"] / df_pivot["발생건수"]

# 4. 각 연도별로 검거율이 가장 높은 범죄유형 찾기
listbox = df_pivot.groupby("연도")["검거율"].idxmax()
max_crime = df_pivot.loc[listbox]

# 5. 각 연도별 최고 검거율 범죄유형의 검거건수를 모두 합산하기
result = max_crime["검거건수"].sum()
result

7799.0

### 문제3

In [ ]:
import pandas as pd
df = pd.read_csv("hr.csv")
# df = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert_v2/main/part4/ch9/hr.csv")

# 1) 만족도 결측치 처리
m = df["만족도"].mean()
df["만족도"] = df["만족도"].fillna(m)

# 2) 근속연수 결측치 처리 (부서와 성과등급 기준 평균값으로 채움)
gm = df.groupby(["부서", "성과등급"])["근속연수"].transform("mean")
gm = gm.astype(int)
df["근속연수"] = df["근속연수"].fillna(gm)

# 3) 연봉 / 근속연수 계산 후 세 번째로 높은 사람의 근속연수 (A)
df["연봉_근속연수"] = df["연봉"] / df["근속연수"]
df_year = df.nlargest(3,'연봉_근속연수')
A = df_year.iloc[-1]["근속연수"]

# 4) 연봉 / 만족도 계산 후 두 번째로 높은 사람의 교육참가횟수 (B)
df["연봉_만족도"] = df["연봉"] / df["만족도"]
df_like = df.nlargest(2,'연봉_만족도')
B = df_like.iloc[-1]["교육참가횟수"]

# 5) 결과 출력
result = A + B
result

7.0

In [ ]:
import pandas as pd
df = pd.read_csv("hr.csv")
# df = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert_v2/main/part4/ch9/hr.csv")

# 1. 만족도 결측치 처리
m = df["만족도"].mean()
df["만족도"] = df["만족도"].fillna(m)

# 2. 근속연수 결측치 처리 (부서와 성과등급 기준 평균값으로 채움)
gm = df.groupby(["부서", "성과등급"])["근속연수"].transform("mean")
gm = gm.astype(int)
df["근속연수"] = df["근속연수"].fillna(gm)

# 3. 연봉 / 근속연수 계산 후 세 번째로 높은 사람의 근속연수 (A)
df["연봉_근속연수"] = df["연봉"] / df["근속연수"]
df_year = df.sort_values(by="연봉_근속연수", ascending=False)
A = df_year.iloc[2]["근속연수"]

# 4. 연봉 / 만족도 계산 후 두 번째로 높은 사람의 교육참가횟수 (B)
df["연봉_만족도"] = df["연봉"] / df["만족도"]
df_like = df.sort_values(by="연봉_만족도", ascending=False)
B = df_like.iloc[1]["교육참가횟수"]

# 5. 결과 출력
result = A + B
result

7.0